In [11]:
import yfinance as yf
import yfinance.shared as shared
import time
import pandas as pd
# from datetime import date, timedelta, datetime
from myUtils import pickle_dump, pickle_load, read_symbols_file # NOQA
from myUtils import drop_symbols_all_NaN, chunked_list # NOQA
from myUtils import yf_download_AdjOHLCV_noAutoAdj
from yf_utils import _2_split_train_val_test, _3_random_slices, _4_perf_ranks
from yf_utils import _5_lookback_slices

verbose = False  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_df_c = 'df_close_clean' 

In [12]:
df_c = pickle_load(path_data_dump, filename_pickled_df_c)
df_train, df_val, df_test = _2_split_train_val_test(df_c)
len_df_train = len(df_train)
len_df_val = len(df_val)
len_df_test = len(df_test)

In [20]:
# _n_samples = 2  # number of tuples to create for iloc start_train:end_train:end_eval
_n_samples = 3  # number of tuples to create for iloc start_train:end_train:end_eval
_days_lookbacks = [30, 120, 60]
_days_eval = 10
_set_syms = 5

In [14]:
# create n_samples of iloc start_train:end_train:end_eval using max value in days_loobacks
max_lookback_slices = _3_random_slices(len_df_train, n_samples=_n_samples, days_lookback=max(_days_lookbacks), days_eval=_days_eval)
sets_lookback_slices = _5_lookback_slices(max_slices=max_lookback_slices, days_lookbacks=_days_lookbacks, verbose=False)
sets_lookback_slices 

[[(332, 347, 357), (317, 347, 357), (287, 347, 357), (227, 347, 357)],
 [(769, 784, 794), (754, 784, 794), (724, 784, 794), (664, 784, 794)],
 [(527, 542, 552), (512, 542, 552), (482, 542, 552), (422, 542, 552)]]

In [25]:
# grp_perf_ranks = {}
grp_most_common_syms = []
# loop thru lists of tuples of start_train:end_train:end_eval, i.e.
#  [[(887, 917, 927), (857, 917, 927), (797, 917, 927)],
#  [(483, 513, 523), (453, 513, 523), (393, 513, 523)]]
for lb_slices in sets_lookback_slices:  
  for lb_slice in lb_slices:  # lb_slice, e.g. (246, 276, 286)
    start_train = lb_slice[0]
    end_train = lb_slice[1]
    start_eval = end_train
    end_eval = lb_slice[2]
    lookback = end_train - start_train
    eval = end_eval - start_eval
    print(f'lb_slices:     {lb_slices}')
    print(f'lb_slice:      {lb_slice}')
    print(f'days lookback: {lookback}')
    print(f'days eval:     {eval}')    
    print(f'start_train:   {start_train}')
    print(f'end_train:     {end_train}')
    # print(f'start_eval:    {start_eval}')
    # print(f'end_eval:      {end_eval}')

    _df = df_train.iloc[start_train:end_train]
    perf_ranks, most_common_syms = _4_perf_ranks(_df, n_top_syms=10)
    # 1 lookback of r_CAGR/UI, r_CAGR/retnStd, r_retnStd/UI
    print(f'perf_ranks: {perf_ranks}')  
    # most common symbols of perf_ranks 
    print(f'most_common_syms: {most_common_syms}')     
    # grp_perf_ranks[lookback] = perf_ranks
    print(f'+++ finish lookback slice {lookback} +++\n')
    grp_most_common_syms.append(most_common_syms)
  print(f'grp_most_common_syms: {grp_most_common_syms}')
  # grp_most_common_syms a is list of lists of tuples of 
  #  the most-common-symbols symbol:frequency cumulated from
  #  each days_lookback  
  print(f'**** finish lookback slices {lb_slices} ****\n')
  # flatten list of lists of (symbol:frequency)
  flat_grp_most_common_syms = [val for sublist in grp_most_common_syms for val in sublist]
  set_most_common_syms = grp_tuples_sort_sum(flat_grp_most_common_syms, reverse=True)
  top_set_syms_n_freq = set_most_common_syms[0:_set_syms]
  top_set_syms = [i[0] for i in top_set_syms_n_freq]  # get keys from key:value pairs
  top_set_syms
  print(f'top {_set_syms} ranked symbols and frequency from set {lb_slices}:\n{top_set_syms_n_freq}')
  print(f'top {_set_syms} ranked symbols from set {lb_slices}:\n{top_set_syms}')  
  print(f'===== finish top {_set_syms} ranked symbols from days_lookback set {lb_slices} =====\n\n')





lb_slices:     [(332, 347, 357), (317, 347, 357), (287, 347, 357), (227, 347, 357)]
lb_slice:      (332, 347, 357)
days lookback: 15
days eval:     10
start_train:   332
end_train:     347


c:\Users\ping\MyDrive\py_files\python\py379\myUtils.py:2516: RuntimeWarning: divide by zero encountered in true_divide
  arr_returns_std_div_arr_UI = arr_returns_std / arr_UI
c:\Users\ping\MyDrive\py_files\python\py379\myUtils.py:2521: RuntimeWarning: divide by zero encountered in true_divide
  arr_CAGR_div_arr_UI = arr_CAGR / arr_UI


perf_ranks: {'period-15': {'r_CAGR/UI': array(['PDI', 'MDGL', 'AVXL', 'STAA', 'SPTN', 'CATO', 'UVV', 'IRDM',
       'WWE', 'SCVL'], dtype=object), 'r_CAGR/retnStd': array(['MDGL', 'AVXL', 'SPTN', 'CATO', 'CYRX', 'WWE', 'IRDM', 'UVV',
       'STAA', 'TNDM'], dtype=object), 'r_retnStd/UI': array(['PDI', 'STAA', 'ADUS', 'MORN', 'UVV', 'PRDO', 'LULU', 'IRDM',
       'SPTN', 'HUBS'], dtype=object)}}
most_common_syms: [('STAA', 3), ('SPTN', 3), ('UVV', 3), ('IRDM', 3), ('PDI', 2), ('MDGL', 2), ('AVXL', 2), ('CATO', 2), ('WWE', 2), ('SCVL', 1), ('CYRX', 1), ('TNDM', 1), ('ADUS', 1), ('MORN', 1), ('PRDO', 1), ('LULU', 1), ('HUBS', 1)]
+++ finish lookback slice 15 +++

lb_slices:     [(332, 347, 357), (317, 347, 357), (287, 347, 357), (227, 347, 357)]
lb_slice:      (317, 347, 357)
days lookback: 30
days eval:     10
start_train:   317
end_train:     347
perf_ranks: {'period-30': {'r_CAGR/UI': array(['TNDM', 'MDGL', 'CRK', 'STAA', 'CYRX', 'WWE', 'PERI', 'MED', 'AMD',
       'BOOM'], dtype=objec

In [17]:
L = [('grape', 100), ('apple', 15), ('grape', 3), ('apple', 10),
    ('apple', 4), ('banana', 3)]

In [18]:
def grp_tuples_sort_sum(L, reverse=True):
  # https://stackoverflow.com/questions/2249036/grouping-python-tuple-list
  # https://stackoverflow.com/questions/10695139/sort-a-list-of-tuples-by-2nd-item-integer-value
  """
  Given a list of tuples of (key:value) such as:
  [('grape', 100), ('apple', 15), ('grape', 3), ('apple', 10),
   ('apple', 4), ('banana', 3)]
  Returns list of grouped-sorted-tuples based on summed-values such as:
  [('grape', 103), ('apple', 29), ('banana', 3)] 

  Args:
      L(list of tuples): list of tuples of key(str):value(int) pairs
      reverse(bool): sort order of summed-values of the grouped tuples,
       default is in descending order.  

  Return:
      grp_sorted_list(list of tuples): list of grouped-sorted-tuples
       based on summed-values such as:
       [('grape', 103), ('apple', 29), ('banana', 3)] 
  """ 

  import itertools
  from operator import itemgetter

  grp_list = []
  l = sorted(L)
  it = itertools.groupby(l, itemgetter(0))

  for key, subiter in it:
    # print(f'key: {key}')
    key_sum = sum(item[1] for item in subiter)
    # print(f'key_sum: {key_sum}')
    grp_list.append((key, key_sum))

  grp_sorted_list = sorted(grp_list, key=itemgetter(1), reverse=reverse)

  return grp_sorted_list  


In [19]:
grp_tuples_sort_sum(L, reverse=True)

[('grape', 103), ('apple', 29), ('banana', 3)]

In [ ]:
# grp_perf_ranks
grp_most_common_syms
# flatten list of lists of (symbols:count)
flat_grp_most_common_syms = [val for sublist in grp_most_common_syms for val in sublist]

sorted_grp_most_common_syms = grp_tuples_sort_sum(flat_grp_most_common_syms, reverse=True)
print(f'len(sorted_grp_most_common_syms): {len(sorted_grp_most_common_syms)}')
sorted_grp_most_common_syms
# top_set_syms = sorted_grp_most_common_syms[0:5]
# top_set_syms = [i[0] for i in top_set_syms]  # get keys from key:value pairs
# top_set_syms 

In [ ]:
top_set_syms = sorted_grp_most_common_syms[0:5]
top_set_syms = [i[0] for i in top_set_syms]  # get keys from key:value pairs
top_set_syms 